In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
import numpy as np

%matplotlib inline

# DATA CLEANING

In [3]:
teis_p = gpd.read_file('../data/TEIS_2022.csv')
teis_p.head( )

,POE,Child ID,DOB,County Name,County SES,Child Phase,Active,Service Coordinator,Notification Date,Parent Consent Date,Referral Source Type Name,Initial Eligibility,Initial Eligibility Date,Initial IFSP Date,Latest IFSP Date,Exit Reason,Exit Date,geometry
0,ET,411812,12/14/2016,Blount,Transitional,Notification,I,Charity Decker,12/16/2016,,DCS,,,,,Unable to contact,12/29/2016,None
1,ET,417398,4/6/2015,Knox,Transitional,Eligibility,I,Jackie Campbell,4/24/2017,5/2/2017,Parent,Ineligible,5/16/2017,,,Ineligible for Part C,5/16/2017,None
2,ET,419400,3/4/2016,Roane,Transitional,Notification,I,Kelly Brackney,6/6/2017,,DCS,,,,,Parent decline,6/30/2017,None
3,ET,397571,5/11/2015,Knox,Transitional,Eligibility,I,Wendy Burton,5/24/2017,6/13/2017,Other,Ineligible,6/16/2017,,,Ineligible for Part C,6/16/2017,None
4,ET,404648,9/23/2015,Knox,Transitional,IFSP,I,Sarah Scott,7/18/2016,7/28/2016,Other,Eligible,8/1/2016,8/16/2016,7/24/2018,618 - Part B eligibility not determined,9/21/2018,None


In [4]:
teis_c = gpd.read_file('../data/TEIS_mon_ref_data.csv')
teis_c.head( )

,POE,Child ID,DOB,County Name,County SES,Child Phase,Active,Service Coordinator,Notification/ Referral Date,Parent Consent Date,...,Service Coordinator Counter,Child Count,2012 Child Count,2013 Child Count,third DOB,Late Referral,Qtr,QTR,Referral Source Category,geometry
0,ET,403339,8/9/2014,Blount,Transitional,Eligibility,A,Kristi Borer,7/1/2016,7/1/16,...,1,1,,,8/9/2017,,1,Qtr 1,#REF!,None
1,FT,404085,7/31/2013,Unicoi,At-Risk,Notification,I,Jennifer Terranera - 45 days,7/1/2016,,...,1,1,,,7/31/2016,1,1,Qtr 1,#REF!,None
2,FT,403623,2/5/2016,Sullivan,Transitional,Eligibility,A,Amy Talbert,7/1/2016,7/20/16,...,1,1,,,2/5/2019,,1,Qtr 1,#REF!,None
3,FT,404157,12/6/2015,Sullivan,Transitional,IFSP,A,Candice Cradic,7/1/2016,7/22/16,...,1,1,,,12/6/2018,,1,Qtr 1,#REF!,None
4,FT,404154,4/18/2015,Hawkins,At-Risk,IFSP,A,Kathy Jeffries,7/1/2016,7/18/16,...,1,1,,,4/18/2018,,1,Qtr 1,#REF!,None


In [5]:
teis_c = teis_c.rename(columns = {'Notification/ Referral Date' : 'Notification Date'})
teis_p = teis_p.rename(columns = {'County SES ' : 'County SES'})

In [6]:
teis_merge = pd.merge(teis_c, teis_p, how='outer')
teis_merge.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 191578 entries, 0 to 191577
Data columns (total 34 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   POE                          191578 non-null  object  
 1   Child ID                     191578 non-null  object  
 2   DOB                          191578 non-null  object  
 3   County Name                  191578 non-null  object  
 4   County SES                   191578 non-null  object  
 5   Child Phase                  191578 non-null  object  
 6   Active                       191578 non-null  object  
 7   Service Coordinator          191578 non-null  object  
 8   Notification Date            191578 non-null  object  
 9   Parent Consent Date          191578 non-null  object  
 10  Referral Source Type Name    191578 non-null  object  
 11  Initial Eligibility          191578 non-null  object  
 12  Initial Eligibility Date     191578 

In [7]:
full_teis= pd.concat([teis_c, teis_p])

In [8]:
teis_clean = full_teis[~full_teis.duplicated(['POE', 'Child ID', 'DOB', 'County Name', 'County SES', 'Child Phase',
       'Active', 'Service Coordinator', 'Notification Date',
       'Parent Consent Date', 'Referral Source Type Name',
       'Initial Eligibility', 'Initial Eligibility Date', 'Initial IFSP Date',
       'Latest IFSP Date', 'Exit Reason', 'Exit Date'])]
teis_clean.info()
#pulling out rows that are the same across all columns that the two dataframes share

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 191578 entries, 0 to 96579
Data columns (total 34 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   POE                          191578 non-null  object  
 1   Child ID                     191578 non-null  object  
 2   DOB                          191578 non-null  object  
 3   County Name                  191578 non-null  object  
 4   County SES                   191578 non-null  object  
 5   Child Phase                  191578 non-null  object  
 6   Active                       191578 non-null  object  
 7   Service Coordinator          191578 non-null  object  
 8   Notification Date            191578 non-null  object  
 9   Parent Consent Date          191578 non-null  object  
 10  Referral Source Type Name    191578 non-null  object  
 11  Initial Eligibility          191578 non-null  object  
 12  Initial Eligibility Date     191578 n

import re
from tqdm.notebook import tqdm

for ind, row in tqdm(teis_clean.iterrows()):
    if re.search('\d{5}', str(row['Exit Date'])):
        teis_clean.loc[ind, 'Exit Date'] = np.NaN
#this is clearing out the weird data that's in the 'Exit Date' column

In [9]:
for ind, row in teis_clean.iterrows():
    if re.search('\d{5}', str(row['Exit Date'])):
        teis_clean.loc[ind, 'Exit Date'] = np.NaN
#this is clearing out the weird data that's in the 'Exit Date' column

NameError: name 're' is not defined

In [ ]:
teis_clean[['DOB', 'Notification Date', 'Parent Consent Date', 'third DOB', 'Exit Date']] = teis_clean[['DOB', 'Notification Date', 'Parent Consent Date', 'third DOB', 'Exit Date']].apply(pd.to_datetime)
teis_clean

In [ ]:
teis_clean = teis_clean.drop(columns = ['2012 Child Count', '2013 Child Count', 'Service Coordinator Counter', 'Referral Count',
                                        'Evaluation Count', 'Eligibility Count', 'IFSP Count', 'Child Count', 'QTR', 
                                       'Referral Source Category'])
teis_clean.head(2)
#there isn't meaningful data in these columns (at least not for the purposes of this project)

In [ ]:
all_teis_dups = teis_clean[teis_clean.duplicated(['Child ID', 'Notification Date'], keep=False)].sort_values('Child ID')
all_teis_dups
#pulling all rows that have 'Child ID' and 'Notification Date' in common to take a look

In [ ]:
num_all_dup = len(np.unique(np.array(all_teis_dups['Child ID'])))
num_all_dup

In [ ]:
county_dups = all_teis_dups[~all_teis_dups.duplicated(['County Name', 'Child ID', 'Notification Date'], keep=False)]
county_dups
#These are instances when the same referral was noted under 2 different counties

In [ ]:
num_county_dup = len(np.unique(np.array(county_dups['Child ID'])))
num_county_dup

In [ ]:
POE_dups = all_teis_dups[~all_teis_dups.duplicated(['POE', 'Child ID', 'Notification Date'], keep=False)]
POE_dups
#These are instances when the same referral was noted under 2 different POEs

In [ ]:
num_POE_dup = len(np.unique(np.array(POE_dups['Child ID'])))
num_POE_dup

In [ ]:
county_status_dups = county_dups[county_dups.duplicated(['Child ID', 'Child Phase', 'Notification Date'], keep=False)]
county_status_dups
#looking at the duplicates that have different counties but the same status

In [ ]:
teis_c[teis_c.duplicated(['Child ID', 'Notification Date'], keep=False)]
#reconfirming that this original dataframe doesn't have any rows that have the same Child ID and Notification Date

In [ ]:
teis_c2 = teis_c.drop(columns = ['POE', 'DOB', 'County Name', 'County SES', 'Child Phase', 'Active', 'Service Coordinator', 
                                 'Parent Consent Date', 'Referral Source Type Name', 'Initial Eligibility', 'Initial Eligibility Date', 'Initial IFSP Date', 'Latest IFSP Date', 'Exit Reason', 'Exit Date',
                                 '2012 Child Count', '2013 Child Count', 'Service Coordinator Counter', 'Referral Count', 
                                 'Evaluation Count', 'Eligibility Count', 'IFSP Count', 'Child Count', 'QTR', 
                                 'Referral Source Category'])
teis_c2

In [ ]:
teis_merge3 = pd.merge(teis_c2, teis_p, how='outer').sort_values(['Child ID'])
teis_merge3

In [ ]:
teis_merge3[['DOB', 'Notification Date', 'Parent Consent Date', 'third DOB', 'Exit Date']] = teis_merge3[['DOB', 'Notification Date', 'Parent Consent Date', 'third DOB', 'Exit Date']].apply(pd.to_datetime)
teis_merge3.info()

In [ ]:
teis_c3 = teis_c.drop(columns = ['2012 Child Count', '2013 Child Count', 'Service Coordinator Counter', 'Referral Count', 
                                 'Evaluation Count', 'Eligibility Count', 'IFSP Count', 'Child Count', 'QTR', 
                                 'Referral Source Category'])
teis_c3

In [ ]:
for ind, row in teis_c3.iterrows():
    if re.search('\d{5}', str(row['Exit Date'])):
        teis_c3.loc[ind, 'Exit Date'] = np.NaN

In [ ]:
teis_c3[['DOB', 'Notification Date', 'Parent Consent Date', 'third DOB', 'Exit Date']] = teis_c3[['DOB', 'Notification Date', 'Parent Consent Date', 'third DOB', 'Exit Date']].apply(pd.to_datetime)
teis_c3

In [ ]:
teis_merge2 = pd.merge(teis_merge3, teis_c3, on=['Child ID', 'Notification Date'], how='left')
teis_merge2.info()
##Need to do this to bring back in the data for the rows in our merge that were present only in teis_c

In [ ]:
teis_merge2['Referral Source Type Name_x'] = teis_merge2['Referral Source Type Name_x'].fillna(teis_merge2['Referral Source Type Name_y'])
teis_merge2['Late Referral_x'] = teis_merge2['Late Referral_x'].fillna(teis_merge2['Late Referral_y'])
teis_merge2['Qtr_x'] = teis_merge2['Qtr_x'].fillna(teis_merge2['Qtr_y'])
teis_merge2['POE_x'] = teis_merge2['POE_x'].fillna(teis_merge2['POE_y'])
teis_merge2['DOB_x'] = teis_merge2['DOB_x'].fillna(teis_merge2['DOB_y'])
teis_merge2['County Name_x'] = teis_merge2['County Name_x'].fillna(teis_merge2['County Name_y'])
teis_merge2['County SES_x'] = teis_merge2['County SES_x'].fillna(teis_merge2['County SES_y'])
teis_merge2['Child Phase_x'] = teis_merge2['Child Phase_x'].fillna(teis_merge2['Child Phase_y'])
teis_merge2['Service Coordinator_x'] = teis_merge2['Service Coordinator_x'].fillna(teis_merge2['Service Coordinator_y'])
teis_merge2['Parent Consent Date_x'] = teis_merge2['Parent Consent Date_x'].fillna(teis_merge2['Parent Consent Date_y'])
teis_merge2['Initial Eligibility_x'] = teis_merge2['Initial Eligibility_x'].fillna(teis_merge2['Initial Eligibility_y'])
teis_merge2['Initial Eligibility Date_x'] = teis_merge2['Initial Eligibility Date_x'].fillna(teis_merge2['Initial Eligibility Date_y'])
teis_merge2['Initial IFSP Date_x'] = teis_merge2['Initial IFSP Date_x'].fillna(teis_merge2['Initial IFSP Date_y'])
teis_merge2['Latest IFSP Date_x'] = teis_merge2['Latest IFSP Date_x'].fillna(teis_merge2['Latest IFSP Date_y'])
teis_merge2['Exit Reason_x'] = teis_merge2['Exit Reason_x'].fillna(teis_merge2['Exit Reason_y'])
teis_merge2['Exit Date_x'] = teis_merge2['Exit Date_x'].fillna(teis_merge2['Exit Date_x'])
teis_merge2.info()
#pulling data from the teis_c3 dataframe into the columns that originated from the teis_merge3 dataframe but had null values

In [ ]:
teis_merge2 = teis_merge2.drop(columns = ['POE_y', 'DOB_y', 'County Name_y', 'County SES_y', 'Child Phase_y', 'Active_y',
                                          'Service Coordinator_y', 'Parent Consent Date_y', 'Referral Source Type Name_y', 
                                          'Initial Eligibility_y', 'Initial Eligibility Date_y', 'Initial IFSP Date_y',
                                          'Latest IFSP Date_y', 'Exit Reason_y', 'Exit Date_y', 'Fiscal Year_y',
                                          'Notification Month_y', 'Tenn Region_y', 'Fiscal Year_y', 'third DOB_y', 
                                          'Late Referral_y', 'Qtr_y'])
teis_merge2.info()
#dropping the no longer needed columns now that we have the cells needed

In [ ]:
teis_merge2 = teis_merge2.rename(columns= {'Fiscal Year_x': 'Fiscal Year',
                                           'Notification Month_x' : 'Notification Month', 
                                           'Tenn Region_x' : 'Tenn Region', 
                                           'Fiscal Year.1_x' : 'Fiscal Year.1',
                                           'third DOB_x' : 'third_DOB',
                                           'Late Referral_x' : 'Late Referral',
                                           'Qtr_x' : 'Qtr',
                                           'POE_x' : 'POE',
                                           'DOB_x' : 'DOB',
                                           'County Name_x' : 'County Name',
                                           'County SES_x' : 'County SES',
                                           'Child Phase_x' : 'Child Phase',
                                           'Active_x' : 'Active',
                                           'Service Coordinator_x' : 'Service Coordinator',
                                           'Parent Consent Date_x' : 'Parent Consent',
                                           'Referral Source Type Name_x' : 'Referral Source Type Name',
                                           'Initial Eligibility_x' : 'Initial Eligibility',
                                           'Initial Eligibility Date_x' : 'Initial Eligibility Date',
                                           'Initial IFSP Date_x' : 'Initial IFSP Date',
                                           'Latest IFSP Date_x' : 'Latest IFSP',
                                           'Exit Reason_x' : 'Exit Reason',
                                           'Exit Date_x' : 'Exit Date'})
teis_merge2.info()
#renaming columns to get rid of the '_x'

In [ ]:
teis_merge2['Year'] = teis_merge2['Notification Date'].dt.year
teis_merge2.head(2)

In [ ]:
all_dups2 = teis_merge2[teis_merge2.duplicated(['Child ID', 'Notification Date'], keep=False)].sort_values('Child ID')
all_dups2
#making sure that there aren't any unanticipated duplicates

# SOCIO ECON QUESTION

In [ ]:
socio = teis_merge2[['POE','Child ID','County Name','County SES']]
socio

In [ ]:
county_group=socio.groupby('POE').count()
county_group

In [ ]:
county_group=socio.groupby('County SES').count()
county_group

In [ ]:
for index, row in socio.iterrows():
    if row['County SES'] == "Distressed":
        socio.loc[index, 'econ_level'] = 5
    elif  row['County SES'] == "At-Risk":
        socio.loc[index, 'econ_level'] = 4 
    elif  row['County SES'] == "Transitional":
        socio.loc[index, 'econ_level'] = 3 
    elif  row['County SES'] == "Competitive":
        socio.loc[index, 'econ_level'] = 2
    elif  row['County SES'] == "Attainment":
        socio.loc[index, 'econ_level'] = 1  
socio

In [ ]:
POE_group=socio.groupby('POE')['econ_level'].sum()/socio.groupby('POE')['econ_level'].count()

In [ ]:
POE_group=POE_group.to_frame().reset_index()

In [ ]:
plt.bar(POE_group.POE,POE_group.econ_level)
plt.title('Average SES by POE')
plt.xlabel('POE')
plt.ylabel('Average SES')

In [ ]:
POE_group

In [ ]:
county_group=socio.groupby('County Name')['econ_level'].sum()/socio.groupby('County Name')['econ_level'].count()

In [ ]:
county_group=county_group.to_frame().reset_index()
county_group

In [ ]:
counties = gpd.read_file('../data/tncounty.shp')

In [ ]:
counties.loc[0, 'geometry']

In [ ]:
print(counties.loc[0, 'geometry'])

In [ ]:
county_group

In [ ]:
counties_SES=pd.merge(counties,county_group, left_on=['NAME'], right_on = ['County Name'], how= 'outer')
counties_SES

In [ ]:
type(counties_SES)

In [ ]:
counties_SES

In [ ]:
fig, ax = plt.subplots(figsize = (20,20))
counties_SES.plot(ax = ax, column = 'econ_level', cmap = 'GnBu',edgecolor = 'black')

for index, row in counties_SES.iterrows():
    plt.annotate(text=row['NAME'], 
                 xy=(row['geometry'].centroid.x, row['geometry'].centroid.y),
                 horizontalalignment='center', fontweight = 'bold')
    
plt.title('SES By County');

In [ ]:
fig, ax = plt.subplots(figsize = (17,17))
counties_SES.plot(ax = ax, column = 'econ_level', legend = True, cmap = 'GnBu',edgecolor = 'black')

for index, row in counties_SES.iterrows():
    plt.annotate(text=row['NAME'], 
                 xy=(row['geometry'].centroid.x, row['geometry'].centroid.y),
                 horizontalalignment='center', fontweight = 'bold')
    
plt.title('SES By County');

# AGE

In [ ]:
teis_merge2.head(5)

In [ ]:
age = teis_merge2[['Notification Date','DOB','POE','Child ID','County Name','County SES']]
age

In [ ]:
type(age)

In [ ]:
age.dtypes

In [ ]:
age

In [ ]:
age['kid_age']=pd.to_datetime(age['Notification Date'])


In [ ]:
age.dtypes

In [ ]:
age

In [ ]:
age['Notification Date'] = (age['Notification Date'] - age['DOB'])


- will want to find the difference between notification date and the DOB.
-Then will want ot remove any values over 3 years differnce.
-Group By (POE, County, referral source, etc)

* Look for patterns in referrals and re-referrals on the basis of the following:
    - Age
    - Referral source
    - Socioeconomic status (County SES)
* TEIS provides services to eligible infants and toddlers from birth to age 3. Are there any POE or service coordinators with high rates of early closure (a child exiting before reaching age 3)? For this question, only consider cases which have reached the IFSP phase.